<a href="https://colab.research.google.com/github/nakib103/pancancer_classification/blob/master/Residual_LSTM_usingLSTMCell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'tensorflow == 2.3.0'

In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
# import packages

import numpy as np
import pandas as pd
import pickle
import math
import gc

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, GRU, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Add
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LambdaCallback

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [11]:
units=32
cell = tf.keras.layers.LSTMCell(units)
input = np.array([[0.2, 0.5], [0.3, 0.7]])
mem = tf.convert_to_tensor(np.zeros((2, units), dtype=np.float32))
carry = tf.convert_to_tensor(np.zeros((2, units), dtype=np.float32))
cell(inputs=input, states=[mem, carry])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



(<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
 array([[-0.01751487, -0.00890173, -0.03140524, -0.01012358, -0.02727371,
          0.02813519,  0.01505922, -0.00239046,  0.01202725, -0.01125331,
         -0.02082691,  0.01820233, -0.01598191,  0.00512828,  0.00348974,
         -0.01504468,  0.02775636, -0.01890246,  0.01016029,  0.01946684,
         -0.02173105, -0.02552926,  0.00318534, -0.01742436, -0.02062202,
          0.0180798 ,  0.00536635,  0.01131215, -0.01880981,  0.01290112,
          0.00642031, -0.012177  ],
        [-0.02463336, -0.01326512, -0.04581995, -0.01400043, -0.03837473,
          0.03994443,  0.02042888, -0.00374378,  0.01672371, -0.01508551,
         -0.03025132,  0.02463639, -0.02332339,  0.00725605,  0.00415502,
         -0.02069422,  0.03965472, -0.02673332,  0.01396547,  0.02605391,
         -0.03020131, -0.03525942,  0.00355791, -0.02370153, -0.0292394 ,
          0.02524588,  0.00742173,  0.01514185, -0.02596478,  0.01797868,
          0.01002288, -0.0

In [15]:
units=32
mem_init = tf.convert_to_tensor(np.zeros((2, units), dtype=np.float32))
carry_init = tf.convert_to_tensor(np.zeros((2, units), dtype=np.float32))

input1 = np.array([[0.2, 0.5], [0.3, 0.7]])
mem, carry = tf.keras.layers.LSTMCell(units)(input1, [mem_init, carry_init])

print(mem, carry)

input2 = np.array([[0.2, 0.5], [0.3, 0.7]])
mem, carry = tf.keras.layers.LSTMCell(units)(input2, carry)


print(mem, carry)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[[ 2.1371369e-03 -3.1672048e-03 -6.4677214e-03  2.2098493e-02
  -1.2521722e-02  1.1506545e-02 -1.4653857e-02 -2.6375610e-02
   2.5979115e-02  3.3197895e-02 -1.5926857e-02  2.0971131e-02
   2.4621960e-02  3.2044929e-02 -1.3317928e-02 -1.6588809e-02
  -4.7974805e-03 -1.0016679e-02  1.1730544e-02 -2.6290722e-02
  -4.9741049e-03 -2.4665892e-02 -4.9845492e-03  2.9143400e-02
  -1.8542824e-02 -9.5590809e-03 -5.6423885e-03  5.9373386e-05
  -1.8050611e-02  2.0590350e-02  1.4222486e-02  6.5053208e-03]
